# Joining and Appending DataFrames in PySpark HW

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [44]:
from utils import Session
from pyspark.sql.functions import *

session = Session("lesson 33")
spark = session.session
spark

ConnectionRefusedError: [Errno 111] Connection refused

## Read in the database

Let cotinue working with our college courses dataframe to get some more insights and practice what we have learned!Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [4]:
import os
path = "Section3/lesson 32/uw-madison-courses/"

df_list = []
for file in os.listdir(path):
    if file.endswith('.csv'):
        df_name = file.split('.')[0]
        df = spark.read.csv(
            path+file,
            inferSchema = True,
            header = True
        )
        df_list.append(df_name)
        df.name = df_name
        # exec(f"{df_name} = {df}")
        exec(df_name + ' = df')

print(df_list)

['courses', 'course_offerings', 'grade_distributions', 'instructors', 'rooms', 'schedules', 'sections', 'subjects', 'subject_memberships', 'teachings']


In [ ]:
# maybe more intuitive but less functional
from typing import Generator

def df_generator() -> Generator:
    for file in os.listdir(path):
        if file.endswith('.csv'):
            df_name = file.split('.')[0]
            df = spark.read.csv(
                path+file,
                inferSchema = True,
                header = True
            )
        
            df.name = df_name
            yield {df_name : df}  

df_list_2 = list(df_generator())

In [6]:
print([i.keys() for i in df_list_2])

[dict_keys(['courses']), dict_keys(['course_offerings']), dict_keys(['grade_distributions']), dict_keys(['instructors']), dict_keys(['rooms']), dict_keys(['schedules']), dict_keys(['sections']), dict_keys(['subjects']), dict_keys(['subject_memberships']), dict_keys(['teachings'])]


In [5]:
courses.limit(5).toPandas()

,uuid,name,number
0,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,Cooperative Education Program,1
1,c070a84f-648f-351d-9499-5d0e30ad02cc,Cooperative Education/Co-op in Life Sciences C...,1
2,e6b4b7ae-0e0b-3aa5-9d77-7fcd90c9cfa3,Cooperative Education Program,1
3,8f63bde1-ff7f-3fe7-9901-862908bf134c,Workshop in Dance Activity,1
4,f3541888-584a-3923-9ce7-6341ff3d84a1,Cooperative Education/Co-op in Agricultural & ...,1


Now check the contents of a few of the dataframses that were read in above.

In [6]:
course_offerings.limit(10).toPandas()

,uuid,course_uuid,term_code,name
0,344b3ebe-da7e-314c-83ed-9425269695fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1092,Cooperative Education Prog
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1082,Cooperative Education Prog
2,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog
3,075da420-5f49-3dd0-93df-13e3c152e1b1,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1114,Cooperative Education Prog
4,2b4e216d-a728-3713-8c7c-19afffc6b2fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1104,Cooperative Education Prog
5,87966a7b-f676-33d0-83d2-acdb67da6790,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1112,Cooperative Education Prog
6,ff511882-5eab-3e7b-a89f-8fbfd1906127,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1134,Cooperative Education Prog
7,b3e26604-fe6e-30df-bc91-b5fa57717a7e,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1084,Cooperative Education Prog
8,e100d196-5e82-32e4-80e9-ac45c07a498c,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1154,Cooperative Education Prog
9,76b9c458-d3c2-38c4-951f-69b6900bd7fe,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1162,Cooperative Education Prog


## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

In [7]:
courses.printSchema()

root
 |-- uuid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- number: integer (nullable = true)



In [8]:
course_offerings.printSchema()

root
 |-- uuid: string (nullable = true)
 |-- course_uuid: string (nullable = true)
 |-- term_code: integer (nullable = true)
 |-- name: string (nullable = true)



In [9]:
rooms.printSchema()

root
 |-- uuid: string (nullable = true)
 |-- facility_code: string (nullable = true)
 |-- room_code: string (nullable = true)



In [10]:
grade_distributions.printSchema()

root
 |-- course_offering_uuid: string (nullable = true)
 |-- section_number: integer (nullable = true)
 |-- a_count: integer (nullable = true)
 |-- ab_count: integer (nullable = true)
 |-- b_count: integer (nullable = true)
 |-- bc_count: integer (nullable = true)
 |-- c_count: integer (nullable = true)
 |-- d_count: integer (nullable = true)
 |-- f_count: integer (nullable = true)
 |-- s_count: integer (nullable = true)
 |-- u_count: integer (nullable = true)
 |-- cr_count: integer (nullable = true)
 |-- n_count: integer (nullable = true)
 |-- p_count: integer (nullable = true)
 |-- i_count: integer (nullable = true)
 |-- nw_count: integer (nullable = true)
 |-- nr_count: integer (nullable = true)
 |-- other_count: integer (nullable = true)



## 1a. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [17]:
step1 = courses.join(
    course_offerings.select(
        col("course_uuid"),
        col("uuid").alias("c_o_uuid")), 
    course_offerings["course_uuid"] == courses["uuid"], 
    how="left").select(["name", "uuid", "c_o_uuid"])

step1.limit(5).toPandas()

,name,uuid,c_o_uuid
0,Cooperative Education Program,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,5da2e500-1b41-3adf-8e03-0614ae7803ab
1,Cooperative Education Program,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,e2b91cd9-7941-319c-a3da-5f682c9d06eb
2,Cooperative Education Program,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,a4f39010-e771-3375-abce-4fa0bdb01139
3,Cooperative Education Program,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,7b8aa2b9-560f-34a6-aa4b-153d1abfb542
4,Cooperative Education Program,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,f84c7f22-aa01-3d7d-90f1-c592f0d56841


In [43]:
step2 = step1.join(
    sections.select(
        col("room_uuid"),
        col("course_offering_uuid")
    ), step1["c_o_uuid"] == sections["course_offering_uuid"], how = "left"
).select(["name", "uuid", "room_uuid"])

step2.limit(5).toPandas()

ConnectionRefusedError: [Errno 111] Connection refused

In [45]:
room_df = step2.join(
    rooms.select(
        "uuid", "room_code", "facility_code"
    ), step2["room_uuid"] == rooms["uuid"], how = "left"
).select(
    rooms.uuid, rooms.room_code, rooms.facility_code, step2.name
).filter(
    "uuid != 'None'"
)

room_df.limit(10).toPandas()


ConnectionRefusedError: [Errno 111] Connection refused

## 1b. Now show same output as above but for only facility number 0469 (facility_code)

## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

## 3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [ ]:
values = [('Monkey',10),('Monkay',36),('Mnky',123), \
          ('Elephant',48),('Elefant',16),('Ellafant',1), \
          ('Hippopotamus',48),('Hipopotamus',16),('Hippo',1)]
zoo = spark.createDataFrame(values,['Animal','age'])
zoo.show()

### Great job! 